<a href="https://colab.research.google.com/github/AntiBron/Machine-learning/blob/main/%22Exercise4_Question_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведен код со ссылкой на набор данных «happy» или «sad», содержащий 80 изображений, 40 счастливых и 40 грустных.
Создайте сверточную нейронную сеть, которая обучается со 100% точностью на этих изображениях, с отменой обучения при достижении точности обучения .999

In [1]:
import tensorflow as tf
import os
import zipfile
#
!wget --no-check-certificate "https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip" \
   -O "/tmp/happy-or-sad.zip"


--2024-05-09 00:24:48--  https://github.com/andrew-veriga/Tensorflow-labs/raw/master/happy-or-sad.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip [following]
--2024-05-09 00:24:48--  https://raw.githubusercontent.com/andrew-veriga/Tensorflow-labs/master/happy-or-sad.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.05s   

2024-05-09 00:24:49 (51.7 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/26

In [2]:

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-02.png', 'happy1-12.png', 'happy1-03.png', 'happy2-10.png', 'happy1-05.png', 'happy2-06.png', 'happy1-06.png', 'happy1-13.png', 'happy2-08.png', 'happy1-10.png']
['sad2-17.png', 'sad2-18.png', 'sad2-03.png', 'sad2-14.png', 'sad1-06.png', 'sad2-13.png', 'sad1-01.png', 'sad1-05.png', 'sad1-19.png', 'sad2-06.png']


In [3]:
import os

train_happy_dir = os.path.join('/tmp/h-or-s/happy')

train_sad_dir = os.path.join('/tmp/h-or-s/sad')

train_happy_names = os.listdir(train_happy_dir)
print(train_happy_names[:10])

train_sad_names = os.listdir(train_sad_dir)
print(train_sad_names[:10])

['happy2-02.png', 'happy1-12.png', 'happy1-03.png', 'happy2-10.png', 'happy1-05.png', 'happy2-06.png', 'happy1-06.png', 'happy1-13.png', 'happy2-08.png', 'happy1-10.png']
['sad2-17.png', 'sad2-18.png', 'sad2-03.png', 'sad2-14.png', 'sad1-06.png', 'sad2-13.png', 'sad1-01.png', 'sad1-05.png', 'sad1-19.png', 'sad2-06.png']


In [4]:

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get('accuracy') > DESIRED_ACCURACY:
            print("\nДостигнута точность 99.9%, обучение прерывается!")
            self.model.stop_training = True
callbacks = myCallback()

# Этот блок кода должен задать и скомпилировать модель. Пожалуйста, имейте в виду, что размер изображений 150 X 150
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

  # Этот блок кода должен создать экземпляр ImageDataGenerator с именем train_datagen
  # и train_generator, вызовом train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

# Используйте target_size  150 X 150.
train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s',
    target_size=(150, 150),
    batch_size=10,
    class_mode='binary'
)

# ожидаемая строка вывода: 'Found 80 images belonging to 2 classes'

# model fitting
history = model.fit(
    train_generator,
    steps_per_epoch=8,  # Число батчей в эпохе (80 изображений в обучающем наборе, batch_size=10)
    epochs=15,
    verbose=1,
    callbacks=[callbacks]
)

# ожидаемая строка вывода: "Достигнута точность 99.9%, обучение прерывается!"


Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 4s 306ms/step - loss: 3.5603 - accuracy: 0.5500
Epoch 2/15
8/8 [==============================] - 4s 486ms/step - loss: 0.8021 - accuracy: 0.4500
Epoch 3/15
8/8 [==============================] - 2s 294ms/step - loss: 0.6971 - accuracy: 0.5875
Epoch 4/15
8/8 [==============================] - 2s 292ms/step - loss: 0.6074 - accuracy: 0.6875
Epoch 5/15
8/8 [==============================] - 2s 301ms/step - loss: 0.4332 - accuracy: 0.8250
Epoch 6/15
8/8 [==============================] - 3s 399ms/step - loss: 0.3217 - accuracy: 0.9000
Epoch 7/15
8/8 [==============================] - 2s 297ms/step - loss: 0.1687 - accuracy: 0.9375
Epoch 8/15
8/8 [==============================] - 2s 297ms/step - loss: 0.1049 - accuracy: 0.9750
Epoch 9/15
8/8 [==============================] - 2s 302ms/step - loss: 0.0618 - accuracy: 0.9875
Epoch 10/15
8/8 [==============================] - ETA: 0s - loss: 0.0289 - ac